In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from src.model.eegnet import EEGNet
from src.datamodule import DataModule, NestedDataSet, DataModuleOld
from src.preprocessing import create_dataset
import pytorch_lightning as pl
import torch

In [ ]:
#Load training module
# The preproccesing code is currently quite messy, still need to clean it up
train, test, val = create_dataset('./src/pickle_df')

In [58]:
# The original method, returns dataloaders which returns a 3d? tensor

dm_old = DataModuleOld(train, test, val, batch_size=16)

# For visualization purposes
x, y = None, None
for (x, y) in dm_old.train_dataloader():
    break

print(x.shape)
print(y.shape)

torch.Size([16, 64, 308])
torch.Size([16])


In [60]:
# Using my custom Dataset class, returning a 4d tensor
dm = DataModule(train, test, val, batch_size=16)

# For visualization purposes
x, y = None, None
for (x, y) in dm.train_dataloader():
    break

print(x.shape)
print(y.shape)

torch.Size([16, 1, 64, 308])
torch.Size([16])


In [61]:
model = EEGNet( chunk_size=308,
                num_electrodes=64,
                dropout=0.5,
                kernel_1=64,
                kernel_2=16,
                F1=8,
                F2=16,
                D=2,
                num_classes=2)

In [62]:
trainer = pl.Trainer()

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [59]:
# Using the original dataloader method, which will error

trainer.fit(model, datamodule=dm_old)


  | Name   | Type       | Params
--------------------------------------
0 | block1 | Sequential | 1.6 K 
1 | block2 | Sequential | 544   
2 | lin    | Linear     | 288   
--------------------------------------
2.4 K     Trainable params
0         Non-trainable params
2.4 K     Total params
0.010     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

RuntimeError: Given groups=1, weight of size [8, 1, 1, 64], expected input[1, 16, 64, 308] to have 1 channels, but got 16 channels instead

In [64]:
# Using my custom loader, which will not error

trainer.fit(model, datamodule=dm)


  | Name   | Type       | Params
--------------------------------------
0 | block1 | Sequential | 1.6 K 
1 | block2 | Sequential | 544   
2 | lin    | Linear     | 288   
--------------------------------------
2.4 K     Trainable params
0         Non-trainable params
2.4 K     Total params
0.010     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

c:\Users\marti\AppData\Local\Programs\Python\Python39\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:430: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
c:\Users\marti\AppData\Local\Programs\Python\Python39\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:430: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

c:\Users\marti\AppData\Local\Programs\Python\Python39\lib\site-packages\pytorch_lightning\trainer\call.py:54: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
